In [2]:
import sympy as sp
from sympy.matrices import Matrix

In [11]:
# Set the number of joints
NUM_JOINTS = 4

# Define symbolic joint variables
joint_symbols = sp.symbols('q1:{}'.format(NUM_JOINTS+1))

# D-H parameters for the robot
dh_parameters = [
    (1, sp.rad(0), 0, joint_symbols[0]),
    (1, sp.rad(0), 0, joint_symbols[1]),
    (1, sp.rad(0), 0, joint_symbols[2]),
    (1, sp.rad(0), 0, joint_symbols[3])
]

# Function to calculate D-H transformation matrix
def dh_transform(a, alpha, d, theta):
    return Matrix([
        [sp.cos(theta), -sp.sin(theta) * sp.cos(alpha), sp.sin(theta) * sp.sin(alpha), a * sp.cos(theta)],
        [sp.sin(theta), sp.cos(theta) * sp.cos(alpha), -sp.cos(theta) * sp.sin(alpha), a * sp.sin(theta)],
        [0, sp.sin(alpha), sp.cos(alpha), d],
        [0, 0, 0, 1]
    ])

# Compute transformation matrices for each joint
transforms = [dh_transform(*params) for params in dh_parameters]

# Compute end-effector transformation matrix
end_effector_transform = sp.simplify(sp.prod(transforms))

# Define end-effector position
end_effector_position = end_effector_transform[:3, 3]

# Calculate Jacobian matrix
num_joints = len(joint_symbols)
jacobian = sp.zeros(3, num_joints)
for i in range(num_joints):
    for j in range(3):
        jacobian[j, i] = sp.diff(end_effector_position[j], joint_symbols[i])

# Display the results
print("End-effector transformation matrix:")
print(end_effector_transform)
print("\nEnd-effector position:")
print(end_effector_position)
print("\nJacobian matrix:")
print(jacobian)


<class 'tuple'>
End-effector transformation matrix:
Matrix([[cos(q1 + q2 + q3 + q4), -sin(q1 + q2 + q3 + q4), 0, cos(q1) + cos(q1 + q2) + cos(q1 + q2 + q3) + cos(q1 + q2 + q3 + q4)], [sin(q1 + q2 + q3 + q4), cos(q1 + q2 + q3 + q4), 0, sin(q1) + sin(q1 + q2) + sin(q1 + q2 + q3) + sin(q1 + q2 + q3 + q4)], [0, 0, 1, 0], [0, 0, 0, 1]])

End-effector position:
Matrix([[cos(q1) + cos(q1 + q2) + cos(q1 + q2 + q3) + cos(q1 + q2 + q3 + q4)], [sin(q1) + sin(q1 + q2) + sin(q1 + q2 + q3) + sin(q1 + q2 + q3 + q4)], [0]])

Jacobian matrix:
Matrix([[-sin(q1) - sin(q1 + q2) - sin(q1 + q2 + q3) - sin(q1 + q2 + q3 + q4), -sin(q1 + q2) - sin(q1 + q2 + q3) - sin(q1 + q2 + q3 + q4), -sin(q1 + q2 + q3) - sin(q1 + q2 + q3 + q4), -sin(q1 + q2 + q3 + q4)], [cos(q1) + cos(q1 + q2) + cos(q1 + q2 + q3) + cos(q1 + q2 + q3 + q4), cos(q1 + q2) + cos(q1 + q2 + q3) + cos(q1 + q2 + q3 + q4), cos(q1 + q2 + q3) + cos(q1 + q2 + q3 + q4), cos(q1 + q2 + q3 + q4)], [0, 0, 0, 0]])


In [24]:
# Create class to calculate jacobian 
class CalcJacobian():
    def __init__(self, dh_params : list, joint_symbols : tuple) -> None:
        self._dh_params = dh_params 
        self._joint_symbols = joint_symbols
        self._transforms = self.compute_transforms()
        self._ee_transform = self.compute_ee_transform(self._transforms)
        self._jacobian = self.calc_jacobian(self._ee_transform)

    # Compute the end effector matrix 
    def compute_transforms(self):
        # Compute transformation matrices for each joint
        for params in self._dh_params:
            print(*params)
        transforms = [self.dh_transform(*params) for params in self._dh_params]
        return transforms

    # Function to calculate D-H transformation matrix
    def dh_transform(a, alpha, d, theta):
        return Matrix([
            [sp.cos(theta), -sp.sin(theta) * sp.cos(alpha), sp.sin(theta) * sp.sin(alpha), a * sp.cos(theta)],
            [sp.sin(theta), sp.cos(theta) * sp.cos(alpha), -sp.cos(theta) * sp.sin(alpha), a * sp.sin(theta)],
            [0, sp.sin(alpha), sp.cos(alpha), d],
            [0, 0, 0, 1]
        ])

    def compute_ee_transform(self, transforms):
        # Compute end-effector transformation matrix
        end_effector_transform = sp.simplify(sp.prod(transforms))
        return end_effector_transform

    def calc_jacobian(self, end_effector_transform):
        # Define end-effector position
        end_effector_position = end_effector_transform[:3, 3]

        # Calculate Jacobian matrix
        num_joints = len(self._joint_symbols)
        jacobian = sp.zeros(3, num_joints)
        for i in range(num_joints):
            for j in range(3):
                jacobian[j, i] = sp.diff(end_effector_position[j], self._joint_symbols[i])

        return jacobian

    def print_results(self):
        # Display the results
        print("End-effector transformation matrix:")
        print(self._end_effector_transform)
        print("\nEnd-effector position:")
        print(self._end_effector_position)
        print("\nJacobian matrix:")
        print(self._jacobian)


##### Execute code here ######

# Set the number of joints
NUM_JOINTS = 4

# Define symbolic joint variables
joint_symbols = sp.symbols('q1:{}'.format(NUM_JOINTS+1))

# D-H parameters for the robot
dh_parameters = [
    (1, sp.rad(0), 0, joint_symbols[0]),
    (1, sp.rad(0), 0, joint_symbols[1]),
    (1, sp.rad(0), 0, joint_symbols[2]),
    (1, sp.rad(0), 0, joint_symbols[3])
]

test1 = CalcJacobian(dh_params=dh_parameters, joint_symbols=joint_symbols)

1 0 0 q1
1 0 0 q2
1 0 0 q3
1 0 0 q4


TypeError: dh_transform() takes 4 positional arguments but 5 were given